In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

4.1

In [ ]:
I = cv2.imread('I.jpg')
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
J = cv2.imread('J.jpg')
J = cv2.cvtColor(J, cv2.COLOR_BGR2GRAY)

difference = cv2.absdiff(I, J)
plt.subplots(1, 3, figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(I, 'gray')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(J, 'gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(difference, 'gray')
plt.axis('off')
plt.show()
def block_optical_flow(I, J, W2, dX, dY):
    u = np.zeros(I.shape).astype(np.float32)
    v = np.zeros(I.shape).astype(np.float32)

    for j in range(W2 * 2, I.shape[0] - W2 * 2):
        for i in range(W2 * 2, I.shape[1] - W2 * 2):
            IO = I[j - W2:j + W2 + 1, i - W2:i + W2 + 1].astype(np.float32)

            x, y = 0 ,0
            most_similar = -1

            for l in range(-dX, dX + 1):
                for k in range(-dY, dY + 1):
                    JO = J[j + l - W2:j + l + W2 + 1, i + k - W2:i + k + W2 + 1].astype(np.float32)
                    dist = np.sum(np.square(IO - JO))
                    if most_similar == -1 or dist < most_similar:
                        most_similar = dist
                        x = k
                        y = l
                u[j, i] = x
                v[j, i] = y
    return u, v
def vis_flow(u, v):
    magnitude, angle = cv2.cartToPolar(u.astype(np.float32), v.astype(np.float32))
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    HSV = np.zeros((u.shape[0], u.shape[1], 3), dtype=np.uint8)
    HSV[:,:,0] = (angle * 90 / np.pi).astype(np.uint8)
    HSV[:,:,1] = magnitude_normalized.astype(np.uint8)
    HSV[:,:,2] = 255

    RGB = cv2.cvtColor(HSV, cv2.COLOR_HSV2RGB)

    plt.imshow(RGB)
    plt.show()
W2 = dX = dY = 3
u, v = block_optical_flow(I, J, W2=5, dX=5, dY=5)

vis_flow(u, v)
I2 = cv2.imread('cm1.png')
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
J2 = cv2.imread('cm2.png')
J2 = cv2.cvtColor(J2, cv2.COLOR_BGR2GRAY)
W2 = dX = dY = 3
u2, v2 = block_optical_flow(I2, J2, W2, dX, dY)

vis_flow(u2, v2)

4.2

In [ ]:
def pyramid(im, max_scale):
    images =[im]
    for k in range(1, max_scale):
        images.append(cv2.resize(images[k - 1], (0, 0), fx=0.5, fy=0.5))
    return images
def pyramid_optical_flow(I_orig, J_orig, W2, dX, dY, num_levels): 
    pyramid_I = pyramid(I_orig, num_levels) 
    pyramid_J = pyramid(J_orig, num_levels)

    current_I = pyramid_I[num_levels - 1]
    current_J = pyramid_J[num_levels - 1]

    u, v = block_optical_flow(current_I, current_J, W2, dX, dY)

    for level in range(num_levels - 2, -1, -1):
        u = cv2.resize(u, (pyramid_I[level].shape[1], pyramid_I[level].shape[0]), interpolation=cv2.INTER_LINEAR) * 2
        v = cv2.resize(v, (pyramid_I[level].shape[1], pyramid_I[level].shape[0]), interpolation=cv2.INTER_LINEAR) * 2

        current_I_original = pyramid_I[level]
        current_J = pyramid_J[level]
        
        I_warped = np.zeros_like(current_I_original, dtype=np.float32)
        rows, cols = current_I_original.shape
        
        for r_idx in range(rows):
            for c_idx in range(cols):
                new_r = int(round(r_idx - v[r_idx, c_idx]))
                new_c = int(round(c_idx - u[r_idx, c_idx]))
                
                new_r = np.clip(new_r, 0, rows - 1)
                new_c = np.clip(new_c, 0, cols - 1)
                
                I_warped[r_idx, c_idx] = current_I_original[new_r, new_c]
        
        du, dv = block_optical_flow(I_warped.astype(np.uint8), current_J, W2, dX, dY)
        
        u += du
        v += dv
        
    return u, v
u3, v3 = pyramid_optical_flow(I, J, W2, dX, dY, 4)

vis_flow(u3, v3)
u4, v4 = pyramid_optical_flow(I2, J2, W2, dX, dY, 4)

vis_flow(u4, v4)
